In [1]:
import re 
import json
import datasets
import pandas as pd
import numpy as np
import os
from report_utils import get_report_sections
from text_utils import regexp, google_translation, funcutf8
pd.set_option('display.max_rows', 400)


In [2]:
def encode_texts(text):
    try:
        return funcutf8(text)
    except:
        return text

In [3]:
def merge_text(text):
    sections = get_report_sections(text)
    texts = []
    for t in sections[1:4]:
        if(type(t) != type(None)):
            if(type(t) != list):
                texts.append(t)
    return ". ".join(texts)

In [33]:
pth_data = "/home/fb198/BA/classification/human_labeled_hf_dataset.json"
original = datasets.Dataset.from_json(pth_data)
original

Using custom data configuration default-65b707b07ffc70c8
Reusing dataset json (/home/fb198/.cache/huggingface/datasets/json/default-65b707b07ffc70c8/0.0.0)


Dataset({
    features: ['text', 'label'],
    num_rows: 165
})

In [34]:
exp_ = original['text']
exp_ = [encode_texts(t) for t in exp_]
exp_ = [merge_text(t) for t in exp_]


In [45]:
exp_
len_ = [len(t.split(" ")) for t in exp_]
len(exp_), len(original), max(len_)

(165, 165, 1170)

In [36]:
data_path = "/home/fb198/BA/classification/new_data.json"
data = datasets.Dataset.from_json(data_path)

label_organs = [l[0] for l in data['label']]
label_disease = [l[1] for l in data['label']]

df = pd.DataFrame(data=([data['text'], label_organs, label_disease])).T
df.columns = ["text", "label_organs", "label_disease"]

dict_organs = {}
dict_disease = {}
idx_to_organ = {}
idx_to_disease = {}
for idx_, label in enumerate(df.label_organs.unique().tolist()):
    dict_organs[label] = idx_
    idx_to_organ[idx_] = label

for idx_, label in enumerate(df.label_disease.unique().tolist()):
    dict_disease[label] = idx_
    idx_to_disease[idx_] = label
df['text'] = df['text'].apply(lambda x: encode_texts(x) )
df['clean_text'] = df['text'].apply(lambda x: merge_text(x) )

df['numerical_organ_label'] = [dict_organs[label] for label in df['label_organs'].tolist()]
df['numerical_disease_label'] = [dict_disease[label] for label in df['label_disease'].tolist()]
df_organs = df[['text', 'numerical_organ_label']]
df_organs.columns = ['text', 'label']
df_disease = df[['text', 'numerical_disease_label']]
df_disease.columns = ['text', 'label']
dataset_organs = datasets.Dataset.from_pandas(df_organs)
dataset_disease = datasets.Dataset.from_pandas(df_disease)

Using custom data configuration default-14e1c5e6c4c32512
Reusing dataset json (/home/fb198/.cache/huggingface/datasets/json/default-14e1c5e6c4c32512/0.0.0)


In [37]:
df

,text,label_organs,label_disease,clean_text,numerical_organ_label,numerical_disease_label
0,Ro - ks Sehr geehrter Herr Kollege! Klinische ...,pancreas,carcinoma,"das 13 cm lange, 4 cm umfassendCholecystektom...",0,0
1,Zugesandt wurden getrennt 1. Polyp im Colon as...,colon,adenoma,Es handelt sich um ein serratiertes Adenom vo...,1,1
2,Klinische Angaben: Kolon-Transversum-Karzinom ...,colon,carcinoma,"1 (Transversum): Ein 24 cm langes, bis maxima...",1,0
3,\t\r\nKlinische Angaben: Proctokolektomie bei ...,colon,adenoma,"1 (Lymphknoten iliocolica): 7x4x 3,5 cm große...",1,1
4,\r\n\r\nKlinische Angaben: Sigmakarzinom. (gt)...,colon,carcinoma,1 (Netz): Ein 24 x7 x5 cm großes lipomatöses ...,1,0
5,\r\n\r\nKlinische Angaben: Rektum-Karzinom Rez...,colon,carcinoma,\t\r\nZustand nach tiefer anteriorer Rectumres...,1,0
6,\r\nKlinische Angaben: Colon Tumor (je) Einges...,colon,carcinoma,"1 (Segmentesektion transversum): 8,5 cm lange...",1,0
7,\r\nKlinische Angaben: hoch sitzendes Rektum-C...,colon,carcinoma,"1 (Rektumresektat): Ein 15 cm langes, 3,5 cm ...",1,0
8,\r\nKlinische Angaben: Kolon-Neoplasie linke F...,colon,adenoma,"1 (Colon): Ein 10,0 cm langes, max. 7,0 cm um...",1,1
9,\r\n\r\nKlinische Angaben: Colon descendens/Si...,colon,carcinoma,"16 cm langes, bis maximal 4cm umfassendes lin...",1,0


In [41]:
merge_text(df.iloc[327]['text'])

''

In [18]:
from collections import Counter
tt = " SONDER_WORD ".join(df['text'].tolist())
tt_ = tt.split(" ")
tt_ = Counter(tt_).most_common()
#tt_[-200:]
tt_new = []
for t in tt_:
    if t[1]<20:
        continue
    if(t[1] > 500):
        continue
    tt_new.append(t)
len(tt_new)

546

In [19]:
tt_new # ('med.', 358), ('Dr.', 357), ('Klinische', 182), ('Beurteilung:', 165), ('Angaben:', 162),  ('Anhalt', 157),
# ('\r\n\r\nKlinische', 114),

[('ein', 484),
 ('im', 480),
 ('A', 441),
 ('eine', 439),
 ('tut', 387),
 ('ich', 381),
 ('weiß', 375),
 ('2,', 360),
 ('med.', 358),
 ('Dr.', 357),
 ('Lymphknoten', 331),
 ('für', 331),
 ('SONDER_WORD', 329),
 ('bis', 325),
 ('Tumor', 322),
 ('nicht', 310),
 ('einem', 309),
 ('wird', 293),
 ('sich', 289),
 ('Bereich', 275),
 ('Ein', 264),
 ('Die', 254),
 ('den', 243),
 ('das', 230),
 ('Fettgewebe', 221),
 ('nach', 218),
 ('dem', 214),
 ('4', 211),
 ('PZpl', 208),
 ('vom', 207),
 ('zu', 202),
 ('Prof.', 202),
 ('aus', 196),
 ('Resektionsrand', 195),
 ('g.', 187),
 ('Klinische', 182),
 ('Schleimhaut', 178),
 ('ohne', 176),
 ('ist', 176),
 ('Nachweis', 175),
 ('li', 172),
 ('sowie', 169),
 ('1,5', 167),
 ('Beurteilung:', 165),
 ('Angaben:', 162),
 ('max.', 162),
 ('Einbettung', 162),
 ('großen', 160),
 ('auch', 160),
 ('zeigt', 159),
 ('Anhalt', 157),
 ('50', 157),
 ('15', 156),
 ('18,', 156),
 ('Marx', 155),
 ('3,', 153),
 ('Kein', 146),
 ('A.', 145),
 ('durch', 144),
 ('Infiltration', 

In [21]:
# german sample texts:
text = 'Klinische Angaben: Kolon-Transversum-Karzinom (je) Eingesandt wurde: 1 (Transversum): Ein 24 cm langes, bis maximal 7 cm umfassendes Dickdarmteilresektat mit anhängendem bis maximal 20 cm breiten Omentumanteil. 9 cm von einem, 12 cm von anderen Resektionsrand sowie 9 cm von mesenterialen Resektionsrand entfernt ein 2,5 x 2 cm großer Tumor mit Makroskopischer Infiltration bis in die Muscularis propria. 1.1,2: Darm Resektionsränder 1.3: Mesenterialer Resektionsrand 1.4-5: Tumor auf Fettgewebe und seröse 1.6: Omentum (makroskopisch ohne Herdbefund) 1.7-11: Lymphknoten tumornah 1.12-14: Lymphknoten tumorfern Beurteilung: Kolonteilresektat mit einem max. 2,5 cm großen, exulcerierten, floride entzündlich überlagerten, drüsig wachsenden, gut bis mäßig differenzierten Adenokarzinom der Dickdarmschleimhaut mit Infiltration bis in die Muscularis propria. Kein Nachweis einer Lymphangio-, Hämangio- oder Perineuralscheideninfiltration. Die beiden Darmresektionsränder und der Meso-Resektionsrand tumorfrei. Im umgebenden Fettgewebe 12 tumorfreie Lymphknoten. Grading: G2 Tumorstadium: pT2 pN0 (0/12); L0 V0 Pn0; R0 Prof. Dr. med. A. Marx N. Petrakopoulou Dr. med. C. Hörner 0621-383 2271 Befundverwertung für wissenschaftliche Zwecke nur mit Genehmigung des Institutsdirektors.\r\n'

In [6]:
sections = get_report_sections(df.iloc[33].text)

In [7]:
sections

(' Kalzifizierende Mesenteritis nach Peritonitis. ',
 ' unter der Bezeichnung "DÃ¼nndarm-Teilresektate" zwei Konglomerat-artige Darmteilresektate. Das erste Resektat ein 10 x 6 x 5cm groÃ\x9fes Konglomerat, die Darmwand mit grau-strÃ¤hniger SchnittflÃ¤che und brauner Serosa, die Mucosa braun-rÃ¶tlich verfÃ¤rbt, unregelmÃ¤Ã\x9fig aufgerauht. Die VerÃ¤nderungen auf anhÃ¤ngendes Weichgewebe Ã¼bergreifend. a, b: Die ResektionsrÃ¤nder. c: ReprÃ¤sentative Querschnitte. d: AuffÃ¤lliges Weichgewebe. Das zweite, ebenfalls Konglomerat-artige Darmresektat 15 x 5 x 5 cm groÃ\x9f, teils bereits erÃ¶ffnet. Prinzipiell gleichartige VerÃ¤nderungen der Darmwand wie bei dem ersten Resektat. Die Darmschleimhaut offenbar an einem Ende in Haut Ã¼bergehend (Stoma?), auch an anderer Stelle eine Ã\x9cberhÃ¤utung nachweisbar, hier zwischen Haut und Darmwand offenbar Weichgewebe befindlich. Das Weichgewebe teils vernarbt, teils erweicht wirkend. e, f: Die ResektionsrÃ¤nder. g: ReprÃ¤sentativeQuerschnitte. h: We

In [8]:
section_names = ['clinical infos:', 'sample infos:', 'desciption:\t', 'diagnosis:\t', 'end:\t\t\t']
for i, section in enumerate(sections):
    print(f'{section_names[i]}\t\t{section}')
    print("\n")


clinical infos:		 Kalzifizierende Mesenteritis nach Peritonitis. 


sample infos:		 unter der Bezeichnung "DÃ¼nndarm-Teilresektate" zwei Konglomerat-artige Darmteilresektate. Das erste Resektat ein 10 x 6 x 5cm groÃes Konglomerat, die Darmwand mit grau-strÃ¤hniger SchnittflÃ¤che und brauner Serosa, die Mucosa braun-rÃ¶tlich verfÃ¤rbt, unregelmÃ¤Ãig aufgerauht. Die VerÃ¤nderungen auf anhÃ¤ngendes Weichgewebe Ã¼bergreifend. a, b: Die ResektionsrÃ¤nder. c: ReprÃ¤sentative Querschnitte. d: AuffÃ¤lliges Weichgewebe. Das zweite, ebenfalls Konglomerat-artige Darmresektat 15 x 5 x 5 cm groÃ, teils bereits erÃ¶ffnet. Prinzipiell gleichartige VerÃ¤nderungen der Darmwand wie bei dem ersten Resektat. Die Darmschleimhaut offenbar an einem Ende in Haut Ã¼bergehend (Stoma?), auch an anderer Stelle eine ÃberhÃ¤utung nachweisbar, hier zwischen Haut und Darmwand offenbar Weichgewebe befindlich. Das Weichgewebe teils vernarbt, teils erweicht wirkend. e, f: Die ResektionsrÃ¤nder. g: ReprÃ¤sentativeQue

In [106]:
## load data
pth_data = "/home/fb198/BA/classification/human_labeled_hf_dataset.json"
data = datasets.Dataset.from_json(pth_data)
len(data['text'])

Using custom data configuration default-65b707b07ffc70c8
Reusing dataset json (/home/fb198/.cache/huggingface/datasets/json/default-65b707b07ffc70c8/0.0.0)


165

In [154]:
## load translated_data
with open("/home/fb198/BA/classification/de_en_translated_filtered_texts.json") as f:
    text_aug = json.load(f)
text_aug = json.loads(text_aug)
df = pd.DataFrame(text_aug['data'])
print(df.shape)
df

(120, 2)


,text,label
0,"The 13 cm long, 4 cm comprehensiveCholecystect...","[pancreas, carcinoma]"
1,It is a sessilic-type serrated adenoma with mi...,"[colon, adenoma]"
2,"1 (transverse): a 24 cm long, up to a maximum ...","[colon, carcinoma]"
3,1 (Lymph nodes iliocolica): 7x4x3.5 cm large a...,"[colon, adenoma]"
4,1 (net): A 24 x7 x5 cm large lipomatous tissue...,"[colon, carcinoma]"
5,A condition after deep anterior rectum resecti...,"[colon, carcinoma]"
6,"1 (segment section transverse): 8.5 cm long, u...","[colon, carcinoma]"
7,1 (rectum resectate): a rectosigmoid resectate...,"[colon, carcinoma]"
8,"1 (Colon): a 10.0 cm long, max.7.0 cm full col...","[colon, adenoma]"
9,"16 cm long, up to a maximum of 4cm comprehensi...","[colon, carcinoma]"


In [155]:
## load translated_data
with open("/home/fb198/BA/classification/en_de_back_translation_filtered_texts.json") as f:
    text_aug = json.load(f)
text_aug = json.loads(text_aug)
df = pd.DataFrame(text_aug['data'])
print(df.shape)
df

(120, 2)


,text,label
0,"Die 13 cm lange, 4 cm umfassende Cholecystekto...","[pancreas, carcinoma]"
1,Es handelt sich um ein sessiles Adenom mit lei...,"[colon, adenoma]"
2,"1 (transversal): ein 24 cm langes, bis maximal...","[colon, carcinoma]"
3,"1 (Lymphknoten Iliocolica): 7x4x3,5 cm großes ...","[colon, adenoma]"
4,1 (Netto): Ein 24 x7 x5 cm großes lipomatische...,"[colon, carcinoma]"
5,Ein Zustand nach tiefer vorderer Rektumresekti...,"[colon, carcinoma]"
6,"1 (Segmentquerschnitt): 8,5 cm lang, bis maxim...","[colon, carcinoma]"
7,1 (Rektumresektat): ein rektosigmoides Resekta...,"[colon, carcinoma]"
8,"1 (Colon): eine 10,0 cm lange, max..7,0 cm Vol...","[colon, adenoma]"
9,"16 cm lang, bis maximal 4cm umfassende linksse...","[colon, carcinoma]"


In [132]:
df.iloc[118].text

'1.................................................................................................................................................(duodenal): Fragmentiert, max..0,5 cm messendes Biopsat.Einbetten in Toto.2.............................................................................................................................................................................................................................................................. .(Antrum): Fragmentiert, max..0,4 cn Biopsat-Messung.Einbetten in toto- 3.(Corpus): Fragmentiert, max..0,4 cm Messung von Biopsat.Einbetten in Toto.Tut mir leid, ich weiß es nicht.1.................................................................................................................................................Tumor, Dysplasie und entzündungsfreie Duodenalschleimhaut ohne Anhalt für eine Sprue.2...............................................................................................................

In [148]:
data_ = data[121:]
pd.DataFrame(data_)
#data_['text']
#data_['label']
#for idx_, text in enumerate(data_['text']):
#    print(data_['label'][idx_])

,text,label
0,\r\n\r\nKlinische Angaben: Mesenchymaler Tumor...,"[stomach, carcinoma]"
1,"\r\n\r\nKlinische Angaben: GERD, fam. Magen-CA...","[stomach, inflammation]"
2,\r\n\t\r\nKlinische Angaben: Dyspeptische Besc...,"[stomach, inflammation]"
3,"\r\n\r\nKlinische Angaben: Pangastritis, Z.n. ...","[stomach, inflammation]"
4,\r\n\r\nKlinische Angaben: Magenkarzinom Antru...,"[stomach, carcinoma]"
5,\r\n\r\nKlinische Angaben: Magenkarzinom; Dign...,"[stomach, carcinoma]"
6,\r\n\r\nKlinische Angaben: Refluxösophagitis. ...,"[stomach, inflammation]"
7,\r\n\r\nKlinische Angaben: NET des Magens sc-w...,"[stomach, NET]"
8,\r\n\r\nKlinische Angaben: Metaplasie des Mage...,"[stomach, inflammation]"
9,\r\n\r\nKlinische Angaben: Adenokarzinom des ö...,"[stomach, carcinoma]"


In [149]:
data_['text'][0]

'\r\n\r\nKlinische Angaben: Mesenchymaler Tumor Oberbauch (rb) Zugesandt wurde: 1 (Arteria hepatica + Truncus coeliacus): Ein weißlich-gräulich knotiges, derb tastbares, 5,0 x 4,7 x 2,8 cm messendes Weichgewebsexzisat. Die Präparateoberfläche schwarz getuscht. Auf den Schnittflächen durch das Präparat zeigen sich mehrere, konfluierend imponierende, weißlich solide Herdbefunde mit fokalen nekrotisch imponierenden Arealen. Die nekrotischen Anteile makroskopisch ca. 15 % des Gesamtvolumens ausmachend. Das weißlich solide Gewebe an allen Seiten < 0,1 cm von der Präparateoberfläche entfernt. 1.1-4: Einbettung von exemplarischen Querschnitten mit Bezug zur getuschten Resektionsfläche sowie den nekrotisch imponierenden Präparateanteilen. 2 (Gallenblase): Ein 7,1 x 3,1 x 2,0 cm messendes Cholezystektomiepräparat. Die Serosa weißlich-gräulich spiegelnd. Die Schleimhaut grünlich samtig. Im Lumen keine Konkremente. Auf den Schnittflächen durch das Präparat kein Herdbefund abgrenzbar. Eingebettet 

In [150]:
data_['label'][0]

['stomach', 'carcinoma']

In [156]:
import torch
torch.cuda.empty_cache()

In [157]:
data_path = "/home/fb198/BA/classification/new_data_filtered.json"
data = datasets.Dataset.from_json(data_path)


Using custom data configuration default-a9bab664444733d7


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/fb198/.cache/huggingface/datasets/json/default-a9bab664444733d7/0.0.0. Subsequent calls will reuse this data.


In [160]:
pth_data = "/home/fb198/BA/classification/human_labeled_hf_dataset.json"
data = datasets.Dataset.from_json(pth_data)

Using custom data configuration default-65b707b07ffc70c8
Reusing dataset json (/home/fb198/.cache/huggingface/datasets/json/default-65b707b07ffc70c8/0.0.0)
